In [63]:
import pandas as pd
import os


In [64]:
# Excel dosyalarının bulunduğu klasör yolu
folder_path = "/Users/celalberkeakyol/Desktop/Data_Deneme/uzun"

# Birleştirilmiş veriler için boş listeler
combined_long_format = []  # 1. Geniş format için (dönemler sütunda)

In [65]:
# Tüm dosyaları tek tek okuyup birleştirme işlemi
for file_name in os.listdir(folder_path):
    if file_name.endswith(".xlsx"):  # Sadece Excel dosyalarını al
        file_path = os.path.join(folder_path, file_name)
        
        # Excel dosyasını oku (örneğin "Bilanço" sheet'i varsayılıyor)
        excel_data = pd.read_excel(file_path, sheet_name="Bilanço")
        
        # Şirket ismini dosya adından çıkar (örneğin, "Company_A.xlsx")
        company_name = os.path.splitext(file_name)[0]
        
  
        
        # 2. Uzun format için veriyi pivot formatına dönüştür
        long_data = excel_data.melt(
            id_vars=["Kalem"],  # "Kalem" sütununu sabit tut
            var_name="Period",  # Dönemler sütun isimlerinden alınır
            value_name="Value"  # Değerler bu sütuna taşınır
        )
        long_data["Company"] = company_name  # Şirket adını ekle
        combined_long_format.append(long_data)




In [66]:
#  Uzun formatlı tabloyu birleştir
long_format_df = pd.concat(combined_long_format, ignore_index=True)
long_format_df["Period"]

0         2024/9
1         2024/9
2         2024/9
3         2024/9
4         2024/9
           ...  
297308    2016/6
297309    2016/6
297310    2016/6
297311    2016/6
297312    2016/6
Name: Period, Length: 297313, dtype: object

# File düzenleme 
Elimizdeki dosyadaki periyot 2024/9 şeklinde bunun yıl ve periyot olarak ayrılması bize avantaj sağlayabilir. 

In [67]:
# 1. Period sütununu split ile ayır
long_format_df[['year', 'quarter']] = long_format_df['Period'].str.split('/', expand=True)

# 2. Company sütunundan '(TRY)' kısmını kaldır
long_format_df['Company'] = long_format_df['Company'].str.replace(r' \(TRY\)', '', regex=True)

# 3. Düzenlenmiş DataFrame'i göster
display(long_format_df)

,Kalem,Period,Value,Company,year,quarter
0,Dönen Varlıklar,2024/9,NaN,TKFEN,2024,9
1,Nakit ve Nakit Benzerleri,2024/9,7.622091e+09,TKFEN,2024,9
2,Gayrimenkul Projeleri Kapsamında Açılan Na...,2024/9,NaN,TKFEN,2024,9
3,Finansal Yatırımlar,2024/9,0.000000e+00,TKFEN,2024,9
4,Teminata Verilen Finansal Varlıklar,2024/9,NaN,TKFEN,2024,9
...,...,...,...,...,...,...
297308,Dönem Net Kar/Zararı,2016/6,3.355969e+07,HEKTS,2016,6
297309,Azınlık Payları,2016/6,NaN,HEKTS,2016,6
297310,Toplam Özkaynaklar,2016/6,1.274539e+08,HEKTS,2016,6
297311,Toplam Kaynaklar,2016/6,2.324601e+08,HEKTS,2016,6


Bir önceki çalışmalarda verilerin tekrar ettiğini bu tekrar eden kalemlerin ise farklı ana başlıktaki kalemlere ait olduğunu gördük 
Bunu çözmek için gerekli kalemleri header olarak sınıflandıralım 
Sonrasında ise Bu kalemlerin altındaki diğer kalemleri sınıflandırarak tekrara düşen verileri başlıklarına göre sınıflandıralım 

In [68]:
# Başlık ve başlığın altındaki verileri ayırabilir miyiz ? 
long_format_df

# 1. Başlık sütunlarını belirleme (boşluk olmayan satırlar)
long_format_df['Is_Header'] = long_format_df['Kalem'].str.startswith(' ') == False

# 2. Başlıkları doldurma: Boşluk olmayan satırları aşağıya taşıyoruz
long_format_df['Header'] = long_format_df['Kalem'].where(long_format_df['Is_Header']).ffill()

# 3. Asıl veriyi temsil eden satırların başındaki boşluğu temizleme
long_format_df['Kalem'] = long_format_df['Kalem'].str.strip()

# 4. Yeni "Grouped_Kalem" sütunu oluşturma (Başlık + Asıl Veri)
long_format_df['Grouped_Kalem'] = long_format_df['Header'] + ' - ' + long_format_df['Kalem']

# 5. Sonucu görüntüleme
display(long_format_df[['Grouped_Kalem', 'Period', 'Value', 'Company']])

,Grouped_Kalem,Period,Value,Company
0,Dönen Varlıklar - Dönen Varlıklar,2024/9,NaN,TKFEN
1,Dönen Varlıklar - Nakit ve Nakit Benzerleri,2024/9,7.622091e+09,TKFEN
2,Dönen Varlıklar - Gayrimenkul Projeleri Kapsam...,2024/9,NaN,TKFEN
3,Dönen Varlıklar - Finansal Yatırımlar,2024/9,0.000000e+00,TKFEN
4,Dönen Varlıklar - Teminata Verilen Finansal Va...,2024/9,NaN,TKFEN
...,...,...,...,...
297308,Özkaynaklar - Dönem Net Kar/Zararı,2016/6,3.355969e+07,HEKTS
297309,Özkaynaklar - Azınlık Payları,2016/6,NaN,HEKTS
297310,Özkaynaklar - Toplam Özkaynaklar,2016/6,1.274539e+08,HEKTS
297311,Özkaynaklar - Toplam Kaynaklar,2016/6,2.324601e+08,HEKTS


In [70]:
long_format_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297313 entries, 0 to 297312
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Kalem          297109 non-null  object 
 1   Period         297313 non-null  object 
 2   Value          149337 non-null  float64
 3   Company        297313 non-null  object 
 4   year           297313 non-null  object 
 5   quarter        297313 non-null  object 
 6   Is_Header      297313 non-null  bool   
 7   Header         297313 non-null  object 
 8   Grouped_Kalem  297109 non-null  object 
dtypes: bool(1), float64(1), object(7)
memory usage: 18.4+ MB


Yukarıdaki info metodu ile görüleceği üzere boş bir verimiz yok. Veriler anlamlı şekilde atomik yapıya ayrıldı. Şimdilik Periyot gibi bazı verileri bilerek tutuyorum sonrasında bu sütunları direkt olarak silebiliriz ama belki ihtiyaç olabilir. 
Bir karışıklık olmaması adına sütunları küçük harfe çevirip sonucu kaydedelim.

In [86]:
long_format_df.columns =  long_format_df.columns.str.lower()

In [88]:
display(long_format_df.head(50))

,kalem,period,value,company,year,quarter,is_header,header,grouped_kalem
0,Dönen Varlıklar,2024/9,NaN,TKFEN,2024,9,True,Dönen Varlıklar,Dönen Varlıklar - Dönen Varlıklar
1,Nakit ve Nakit Benzerleri,2024/9,7.622091e+09,TKFEN,2024,9,False,Dönen Varlıklar,Dönen Varlıklar - Nakit ve Nakit Benzerleri
2,Gayrimenkul Projeleri Kapsamında Açılan Nakit ...,2024/9,NaN,TKFEN,2024,9,False,Dönen Varlıklar,Dönen Varlıklar - Gayrimenkul Projeleri Kapsam...
3,Finansal Yatırımlar,2024/9,0.000000e+00,TKFEN,2024,9,False,Dönen Varlıklar,Dönen Varlıklar - Finansal Yatırımlar
4,Teminata Verilen Finansal Varlıklar,2024/9,NaN,TKFEN,2024,9,False,Dönen Varlıklar,Dönen Varlıklar - Teminata Verilen Finansal Va...
5,Ticari Alacaklar,2024/9,5.558219e+09,TKFEN,2024,9,False,Dönen Varlıklar,Dönen Varlıklar - Ticari Alacaklar
6,Finans Sektörü Faaliyetlerinden Alacaklar,2024/9,NaN,TKFEN,2024,9,False,Dönen Varlıklar,Dönen Varlıklar - Finans Sektörü Faaliyetlerin...
7,Türkiye Cumhuriyet Merkez Bankası Hesabı,2024/9,NaN,TKFEN,2024,9,False,Dönen Varlıklar,Dönen Varlıklar - Türkiye Cumhuriyet Merkez Ba...
8,Diğer Alacaklar,2024/9,2.492220e+08,TKFEN,2024,9,False,Dönen Varlıklar,Dönen Varlıklar - Diğer Alacaklar
9,Müşteri Sözleşmelerinden Doğan Varlıklar,2024/9,1.953020e+09,TKFEN,2024,9,False,Dönen Varlıklar,Dönen Varlıklar - Müşteri Sözleşmelerinden Doğ...


In [90]:
#Kaydetme aşaması 
output_file = "cleaned_long_format.xlsx"  # Kaydedilecek dosya adı
long_format_df.to_excel(output_file, index=False)

print(f"Değişiklikler kalıcı hale getirildi ve dosya '{output_file}' olarak kaydedildi.")

Değişiklikler kalıcı hale getirildi ve dosya 'cleaned_long_format.xlsx' olarak kaydedildi.
